In [33]:
import pandas as pd
from collections import Counter

In [4]:
data = pd.read_csv("NCAA_AP_TOP_25.csv")
data2 = pd.read_csv("NCAA_BRACKET.csv")

In [28]:
data2.head()

,YEAR,REGION,ROUND,SEED,TEAM,SCORE,O_SEED,O_NAME,O_SCORE
0,2002,East Regional,1,1,Maryland,85,16,Siena,70
1,2002,East Regional,1,8,Wisconsin,80,9,St. John's (NY),70
2,2002,East Regional,1,5,Marquette,69,12,Tulsa,71
3,2002,East Regional,1,4,Kentucky,83,13,Valparaiso,68
4,2002,East Regional,1,6,Texas Tech,68,11,Southern Illinois,76


In [43]:
def winner(seed1, seed2, score1, score2):
# print seed1, seed2, score1, score2
    if score1 > score2:
        return seed2 - seed1
    else:
        return seed1 - seed2

In [44]:
Counter(data2.apply(lambda x: winner(x.SEED, x.O_SEED, x.SCORE, x.O_SCORE), axis = 1))

Counter({-13: 3,
         -11: 7,
         -10: 2,
         -9: 11,
         -8: 25,
         -7: 32,
         -6: 4,
         -5: 32,
         -4: 13,
         -3: 42,
         -2: 7,
         -1: 67,
         0: 11,
         1: 80,
         2: 11,
         3: 78,
         4: 34,
         5: 63,
         6: 4,
         7: 65,
         8: 68,
         9: 52,
         10: 2,
         11: 57,
         12: 3,
         13: 53,
         15: 56})